# Linear Algebra

## Systems of Linear Equations
A system of linear equations is a formulation of multiple linear equations.

They can be written as $Ax=b$ by grouping the values that scale the xs in the equations into a matrix and then taking the right hand side of the equations and the xs into two vectors.

Solving a system of linear equations means finding values of xs that satisfy the Ax=b. There are potentially 0 solutions, 1 solutions or infinite solutions.

## Matrices
A matrix is a transformation (linear map) once bases are chosen that can be applied to a vector.

Matrix multiplication means applying that transformation to a vector or composing two transformations.

## Solving
Elementary row operations are addition of a scalar multiple of a row, subtraction of a scalar multiple of a row, swapping and multiplying by a non-zero scalar.

Solutions can be:
- Unique when there is a single solution.
- Nonexistent when the system is inconsistent.
- Infinite when there is a dependent system.

In [3]:
data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
data.sort(reverse=True)
print("[" + "\n".join(str(x) for x in data) + "]")

[[7, 8, 9]
[4, 5, 6]
[1, 2, 3]]


In [ ]:
from operator import add, mul, sub
EPS = 1e-12

class Matrix():

    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __setitem__(self, index, value):
        self.data[index] = value

    def __str__(self):
        return "[" + '\n'.join(str(row) for row in self.data) + "]"

    def shape(self):
        return len(self.data), len(self.data[0])

    def compute_row_echelon_form(self):
        rows, cols = self.shape()
        A = [row[:] for row in self.data]

        row = 0
        col = 0

        while row < rows and col < cols:
            # find pivot
            pivot = False
            while col < cols and not pivot:
                for r in range(row, rows):
                    if abs(A[r][col]) > EPS:
                        if r != row:
                            A[r], A[row] = A[row], A[r]
                        pivot = True
                        break
                if not pivot:
                    col += 1

            # subtract from rows below
            for r in range(row + 1, rows):
                if abs(A[r][col]) > EPS:
                    diff = A[r][col]/A[row][col]
                    A[r] = [x - diff * y for x, y in zip(A[r], A[row])]

            row += 1
            col += 1

        return Matrix(A)

    def compute_reduced_row_echelon_form_helper(self):
        # Add setting all rows in pivot column to 0
        row_echelon_form = self.compute_row_echelon_form()
        rows, _ = row_echelon_form.shape()
        pivots = []
        for r in range(rows):
            row = row_echelon_form.data[r]
            pivot = next((i for i, x in enumerate(row) if abs(x) > EPS), None)

            if pivot != None:
                pivots.append(pivot)
                row_echelon_form.data[r] = [x / row[pivot] for x in row]
                row_echelon_form.data[r] = [
                    0.0 if abs(x) < EPS else x for x in row_echelon_form.data[r]
                ]
                for ir in range(rows):
                    if ir != r:
                        diff = row_echelon_form.data[ir][pivot]
                        row_echelon_form.data[ir] = list(
                            map(
                                sub,
                                row_echelon_form.data[ir],
                                [x * diff for x in row_echelon_form.data[r]],
                            )
                        )
                        row_echelon_form.data[ir] = [
                            0.0 if abs(x) < EPS else x
                            for x in row_echelon_form.data[ir]
                        ]
        return row_echelon_form, pivots

    def compute_reduced_row_echelon_form(self):
        rref, _ = self.compute_reduced_row_echelon_form_helper()
        return rref

    def find_pivots(self):
        _, pivots = self.compute_reduced_row_echelon_form_helper()
        return pivots

    def get_rank(self):
        return len(self.find_pivots())

    def is_independent(self):
        rank = self.get_rank()
        _, cols = self.shape()
        return rank == cols

    def get_basis(self):
        pivots = self.find_pivots()
        return Matrix([[row[p] for p in pivots] for row in self.data])

    def solve_via_gauss(self):
        rref = self.compute_reduced_row_echelon_form()
        b = Matrix([[r[-1]] for r in rref.data])
        A = Matrix([r[:-1] for r in rref.data])
        print(f"A: {A}\nb:{b}\n")
        pivots = self.find_pivots()
        rows, cols = A.shape()
        if len(pivots) == cols:
            pass
            # full rank so just make equal to 0
        else:
            not_pivots = list(set(range(cols)) - set(pivots))
            for c in not_pivots:
                new_row = [0] * cols
                new_row[c] = -1
                A.data.insert(c, new_row)
                # maybe insert into b too?
            solution_columns = Matrix([[r[c] for c in not_pivots] for r in A.data])
            print(f"solution: {solution_columns}\n")
        return 

    def num_solutions(self):
        rref = self.compute_reduced_row_echelon_form()
        _, cols = rref.shape()

        for row in rref:
            pivot = next((i for i, x in enumerate(row) if abs(x) > EPS), None)
            if pivot == cols - 1:
                return 0
        pivots = self.find_pivots()
        if set(pivots) == set(range(cols - 1)):
            # unique solution
            return 1
        else:
            # infinite solutions
            return -1

    def get_column_space(self):
        # TODO
        pass

    def get_null_space(self):
        # TODO
        pass

    def compute_determinant(self):
        # TODO
        pass

    def apply_mapping(self, vector):
        _, cols = self.shape()
        vrows, vcols = vector.shape()
        if vcols != 1:
            raise TypeError(f"Vectors can only have one column")

        if vrows != cols:
            raise TypeError(f"Dimensions must match: {self.shape(), vector.shape()}")

        flat_vector = [x for xs in vector.data for x in xs]

        new_vector = []
        for r in self.data:
            new_vector.append([sum(map(mul, flat_vector, r))])

        return Matrix(new_vector)

    def compute_kernel_basis(self):
        # TODO
        pass

    def compute_image_basis(self):
        # TODO
        pass

    def compute_affine_solution(self, vector):
        # TODO
        pass

In [26]:
matrix = Matrix([[1, 2, 5], [3, 4, 11]])
print(matrix.solve_via_gauss())

matrix = Matrix([[1, 1, 2], [2, 2, 5]])
print(matrix.solve_via_gauss())

matrix = Matrix([[1, 1, 2], [2, 2, 4]])
print(matrix.solve_via_gauss())

matrix = Matrix([[0, 1, 3], [2, 4, 10]])
print(matrix.solve_via_gauss())

matrix = Matrix([[1, 0, 2, 3], [0, 1, -1, 1]])
print(matrix.solve_via_gauss())

A: [[1.0, 0.0]
[0.0, 1.0]]
b:[[1.0]
[2.0]]

None
A: [[1.0, 1.0]
[0.0, 0.0]]
b:[[0.0]
[1.0]]

None
A: [[1.0, 1.0]
[0.0, 0.0]]
b:[[2.0]
[0.0]]

solution: [[1.0]
[-1]
[0.0]]

None
A: [[1.0, 0.0]
[0.0, 1.0]]
b:[[-1.0]
[3.0]]

None
A: [[1.0, 0.0, 2.0]
[0.0, 1.0, -1.0]]
b:[[3.0]
[1.0]]

solution: [[2.0]
[-1.0]
[-1]]

None


In [ ]:

matrix = Matrix([[1, 0], [0, 1]])

vector = Matrix([[2],[3]])

print(matrix.apply_mapping(vector))

In [ ]:
data = [[1, 2], [3, 4]]


matrix = Matrix(data)
print("\n", matrix.compute_row_echelon_form())
print('\n', matrix.compute_reduced_row_echelon_form())
print('\n', matrix.find_pivots())
print("\n", matrix.get_basis())

## Vector Spaces
A real valued vector space is an abelian group: $V=(\mathcal{V},+,\cdot)$, two operations $+:\mathcal{V}\times\mathcal{V} \rightarrow \mathcal{V}$ and $\cdot:\mathbb{R}\times\mathcal{V} \rightarrow \mathcal{V}$ matrix multiplication is not defined.

## Linear Independence
- Columns are linearly independent when no column is redundant (cannot be described through a combination of the other columns). $0 = \sum_{i=1}^{k}\lambda_i x_i$
- Linearly independent columns can be found using pivot columns which form a basis for the column space.

## Basis and Rank
- A basis is a set of linearly independent columns that can be used to generate all vectors in a set. It is minimal generating set.
- Span is used to indicate the set that the basis vectors cover. $V=span[\mathcal{A}]$
- A rank is the number of linearly independent columns in a matrix
- Every vector space possesses a basis
- The dimension of a vector space is the number of basis vectors

The **rank-nullity theorem** means $dim(Im(\Phi)) + dim(ker(\Phi)) = dim(V)$

## Linear Mappings
- LM or homomorphism is when: $\forall x,y \in V, \forall \lambda, \psi \in \mathbb{R}: \Phi(\lambda x + \psi y) = \lambda\Phi(x) + \psi\Phi(y)$
- Can be represented as matrices that transform vectors from one space to another.

Linear Mapping $\Phi$, and spaces $\mathcal{V} \rightarrow \mathcal{W}$
- Injective: $\forall x,y \in \mathcal{V} : \Phi(x) = \Phi(y) \implies x = y$
- Surjective: $\Phi(\mathcal{V}) = \mathcal{W} $
- Bijective: injective and surjective, bijective linear mapping have an inverse

Cases
- Isomorphism: $\Phi: \mathcal{V} \rightarrow \mathcal{W}$; linear and bijective
- Endomorphism: $\Phi: \mathcal{V} \rightarrow \mathcal{V}$; linear
- Automorphism: $\Phi: \mathcal{V} \rightarrow \mathcal{V}$; linear and bijective

- Vector spaces with the same dimensions can be transformed into one another without any loss. (Assuming a finite dimensional space and an isomorphism after choosing bases).
- The conjunction of two linear mappings is also linear.

A Kernel is the set of vectors that goes to 0 under the transformation.

An image is the set of vectors that can be reached in the target space from the origin space.


## Affine Spaces
- A translation of a linear subspace.

## Relation to Machine Learning
Data can be represented as vectors and datasets as matrices. When applied to a model a vector can be considered a single row of data and many rows as a dataset.

Rank is important for learning because it shows potential redundancy.